In [ ]:
# eigen_calc.py

import numpy as np

def get_dimensions(matrix):
    """Return [rows, cols] for a list-of-lists matrix."""
    return [len(matrix), len(matrix[0])]

def find_determinant(matrix, excluded=1):
    """
    Recursive determinant for a pure-Python matrix.
    'excluded' is the multiplier from the previous cofactor.
    """
    dims = get_dimensions(matrix)
    # base case 2×2
    if dims == [2, 2]:
        return excluded * (matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0])
    # recursive case
    det = 0
    for j in range(dims[1]):
        # build submatrix by removing row 0 and column j
        sub = [
            [matrix[i][k] for k in range(dims[1]) if k != j]
            for i in range(1, dims[0])
        ]
        cofactor = matrix[0][j] * ((-1) ** j)
        det += cofactor * find_determinant(sub, 1)
    return det

def list_multiply(p1, p2):
    """
    Multiply two polynomials p1 and p2, given as coefficient lists.
    E.g. [a0,a1] represents a0 + a1*x.
    """
    result = [0] * (len(p1) + len(p2) - 1)
    for i, a in enumerate(p1):
        for j, b in enumerate(p2):
            result[i+j] += a * b
    return result

def list_add(p1, p2, sub=1):
    """
    Add two polynomials p1 and p2 termwise, with optional sign on p2.
    """
    n = max(len(p1), len(p2))
    p1 = p1 + [0]*(n - len(p1))
    p2 = p2 + [0]*(n - len(p2))
    return [p1[i] + sub*p2[i] for i in range(n)]

def determinant_equation(matrix, excluded=[1, 0]):
    """
    Build characteristic polynomial coefficients for a 'matrix' whose
    entries are [a, -b] lists representing (a - b*λ).
    Returns a list [c0, c1, ..., cn] for c0 + c1*λ + ... + cn*λ^n.
    """
    dims = get_dimensions(matrix)
    # base 2×2: |a - λ   b     |
    #          |c     d - λ| ⇒ (a-λ)(d-λ) - b*c
    if dims == [2, 2]:
        # matrix = [[ [a,-1], [b,0] ], [ [c,0], [d,-1] ]]
        m00, m01 = matrix[0]
        m10, m11 = matrix[1]
        # (m00[0] + m00[1]*λ)*(m11[0] + m11[1]*λ) - (m01[0] + m01[1]*λ)*(m10[0] + m10[1]*λ)
        poly1 = list_multiply(m00, m11)
        poly2 = list_multiply(m01, m10)
        return list_add(poly1, poly2, sub=-1)
    # general NxN via cofactor expansion
    eqs = []
    for j in range(dims[1]):
        # extract factor [a,-b] at (0,j)
        factor = matrix[0][j]
        # build submatrix removing row 0 and column j
        sub = [
            [row[k] for k in range(dims[1]) if k != j]
            for row in matrix[1:]
        ]
        # recursive characteristic equation for sub
        sub_eq = determinant_equation(sub, excluded=[1,0])
        # multiply polynomial by factor and sign
        term = list_multiply(factor, sub_eq)
        if j % 2 == 1:
            term = [-c for c in term]
        eqs.append(term)
    # sum all term polynomials termwise
    result = [0] * max(len(t) for t in eqs)
    for t in eqs:
        result = list_add(result, t)
    return result

def identity_matrix(dimensions):
    """Return identity matrix as list-of-lists."""
    return [[1 if i == j else 0 for j in range(dimensions[1])]
            for i in range(dimensions[0])]

def characteristic_equation(matrix):
    """
    Turn a numeric matrix into a 'symbolic' matrix whose entries are [a,-b]
    representing (a - b*λ).
    """
    dims = get_dimensions(matrix)
    I = identity_matrix(dims)
    return [
        [[matrix[i][j], -I[i][j]] for j in range(dims[1])]
        for i in range(dims[0])
    ]

def find_eigenvalues(matrix):
    """
    Compute eigenvalues from characteristic polynomial roots.
    """
    # build coefficient list [c0, c1, ..., cn]
    coeffs = determinant_equation(characteristic_equation(matrix))
    # numpy.roots wants highest-power first
    poly = coeffs[::-1]
    return np.roots(poly)

def compare_with_numpy(matrix):
    """
    Return (custom_eigenvalues, numpy_eigenvalues).
    """
    custom = find_eigenvalues(matrix)
    numpy_vals, _ = np.linalg.eig(np.array(matrix, dtype=float))
    return custom, numpy_vals

if __name__ == "__main__":
    A = [[6, 1, -1],
         [0, 7,  0],
         [3, -1, 2]]

    custom_vals, numpy_vals = compare_with_numpy(A)
    print("Custom eigenvalues:", custom_vals)
    print("NumPy eigenvalues: ", numpy_vals)


Custom eigenvalues: [7. 5. 3.]
NumPy eigenvalues:  [5. 3. 7.]
